In [1]:
# --- Imports ---
import os
import glob
import re
from datetime import datetime
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from PIL import Image, ImageDraw, ImageFont
import imageio
import warnings
import csv
import platform # To help find system fonts
from typing import List, Optional, Tuple # For type hinting

# --- Add Geopandas and Rasterio features ---
try:
    import geopandas as gpd
    from rasterio.features import rasterize
    from rasterio.transform import from_bounds
    GEOPANDAS_AVAILABLE = True
except ImportError:
    GEOPANDAS_AVAILABLE = False
    print("WARNING: Geopandas not found. Country masking feature will be disabled.")


# --- Helper function to parse date from filename ---
def get_date_from_filename(filename):
    """Extracts YYYY_MM from common GEE export filenames."""
    # Pattern: _YYYY_MM.tif
    match = re.search(r'_(\d{4})_(\d{2})\.tif$', os.path.basename(filename))
    if match:
        year, month = map(int, match.groups())
        if 1 <= month <= 12: return datetime(year, month, 1)
        else: print(f"Warning: Invalid month '{month}' in _YYYY_MM: {filename}. Skip."); return None
    # Pattern: YYYYMM.tif
    match_alt = re.search(r'(\d{4})(\d{2})\.tif$', os.path.basename(filename))
    if match_alt:
        year, month = map(int, match_alt.groups())
        if 1 <= month <= 12: return datetime(year, month, 1)
        else: print(f"Warning: Invalid month '{month}' in YYYYMM: {filename}. Skip."); return None
    # Pattern: YYYY-MM.tif
    match_alt2 = re.search(r'(\d{4})-(\d{2})\.tif$', os.path.basename(filename))
    if match_alt2:
        year, month = map(int, match_alt2.groups())
        if 1 <= month <= 12: return datetime(year, month, 1)
        else: print(f"Warning: Invalid month '{month}' in YYYY-MM: {filename}. Skip."); return None
    # Fallback
    print(f"Warning: Could not parse date from filename: {filename} (tried patterns). Skipping.")
    return None

# --- Helper function to find fonts ---
def find_system_font(font_names):
    """ Helper to find a usable TTF/OTF font file """
    system = platform.system(); common_paths = []
    if system == "Windows": common_paths = [os.path.join(os.environ.get("SystemRoot", "C:\\Windows"), "Fonts")]
    elif system == "Linux": common_paths = ["/usr/share/fonts/truetype/dejavu","/usr/share/fonts/truetype/msttcorefonts","/usr/share/fonts/truetype/liberation","/usr/share/fonts/truetype","/usr/local/share/fonts",os.path.expanduser("~/.fonts")]
    elif system == "Darwin": common_paths = ["/System/Library/Fonts","/Library/Fonts",os.path.expanduser("~/Library/Fonts")]
    common_paths.append(".")
    for name in font_names:
        try: ImageFont.truetype(name, 10); print(f"  Found font '{name}' by name."); return name
        except IOError: pass
        for directory in common_paths:
            if not os.path.isdir(directory): continue
            base_name_lower = name.lower()
            try:
                for fname in os.listdir(directory):
                    fname_lower = fname.lower()
                    if fname_lower.startswith(base_name_lower) and (fname_lower.endswith(".ttf") or fname_lower.endswith(".otf")):
                        potential_path = os.path.join(directory, fname)
                        if os.path.isfile(potential_path):
                             try: ImageFont.truetype(potential_path, 10); print(f"  Found font at: {potential_path}"); return potential_path
                             except IOError: continue
            except OSError: continue
    return None

# --- Main Function ---
def create_nightlight_timelapse_and_graph(
    input_folder: str,
    output_path_base: str,
    output_format: str = 'gif',
    cmap_name: str = 'plasma',
    fps: int = 6,
    normalize_animation: bool = False,
    mask_outside_country: bool = True,
    country_boundary_shapefile_path: str = None,
    target_country_name: str = None,
    outside_mask_color: tuple = (0, 0, 0, 0),
    mp4_crf: int = 23,
    text_size_category: str = 'medium',
    font_path: str = None,
    graph_title: str = 'Average Night Light Intensity Over Time',
    watermark_text: str = 'My Custom Watermark',
    skip_dark_frames: bool = False,
    dark_frame_threshold: float = 0.0,
    exclude_months: Optional[List[int]] = None, # <<< New option: List of months (1-12) to exclude from video
    add_progress_bar: bool = True,
    progress_bar_height: int = 5,
    progress_bar_color_bg: tuple = (50, 50, 50, 180),
    progress_bar_color_fg: tuple = (255, 255, 255, 200),
    debug_masking: bool = False
    ):
    """
    Creates timelapse, graph, and CSV with multiple night light metrics from GeoTIFFs,
    optionally masking areas outside a specified country boundary, skipping dark frames,
    skipping specified months in the animation, and adding a progress bar.

    Args:
        input_folder (str): Path to the folder containing GeoTIFF files.
        output_path_base (str): Base path and filename for output files (e.g., 'output/country_lights').
        output_format (str, optional): Animation format ('gif' or 'mp4'). Defaults to 'gif'.
        cmap_name (str, optional): Matplotlib colormap name. Defaults to 'plasma'.
        fps (int, optional): Frames per second for animation. Defaults to 6.
        normalize_animation (bool, optional): Normalize colors based on 0.5-99.5 percentile of lit pixels
                                             if True, else use fixed 0-max range. Defaults to False.
        mask_outside_country (bool, optional): Mask areas outside the country. Defaults to True.
        country_boundary_shapefile_path (str, optional): Path to country boundaries shapefile.
        target_country_name (str, optional): Name of the country to keep (case-insensitive).
        outside_mask_color (tuple, optional): RGBA color (0-255) for masked areas. Defaults to (0,0,0,0).
        mp4_crf (int, optional): CRF for MP4 encoding. Defaults to 23.
        text_size_category (str, optional): Relative text size ('small', 'medium', 'large'). Defaults to 'medium'.
        font_path (str, optional): Path to a specific .ttf or .otf font file. Defaults to None.
        graph_title (str, optional): Title for the graph. Defaults to 'Average Night Light Intensity Over Time'.
        watermark_text (str, optional): Watermark text (top-left). None or "" disables. Defaults to 'My Custom Watermark'.
        skip_dark_frames (bool, optional): Skip frames if sum_lit <= dark_frame_threshold. Defaults to False.
        dark_frame_threshold (float, optional): Threshold for sum_lit to skip dark frames. Defaults to 0.0.
        exclude_months (Optional[List[int]], optional): A list of month numbers (1-12) to exclude
                                                      from the generated animation frames. Data for these months
                                                      will still be in the CSV and graph. Defaults to None (no exclusion).
        add_progress_bar (bool, optional): Add a progress bar to animation frames. Defaults to True.
        progress_bar_height (int, optional): Progress bar height in pixels. Defaults to 5.
        progress_bar_color_bg (tuple, optional): RGBA color for progress bar background. Defaults to (50,50,50,180).
        progress_bar_color_fg (tuple, optional): RGBA color for progress bar foreground. Defaults to (255,255,255,200).
        debug_masking (bool, optional): Enable extra debug prints for masking. Defaults to False.
    """

    print(f"Starting analysis for folder: {input_folder}")
    print(f"Output base: {output_path_base}")
    print(f"Format: {output_format}, FPS: {fps}, Colormap: {cmap_name}")
    print(f"Normalize Animation Frames: {normalize_animation}")
    print(f"Mask Outside Country: {mask_outside_country}")
    if mask_outside_country: print(f"Target Country Name: '{target_country_name}'")
    if output_format.lower() == 'mp4': print(f"MP4 CRF Value: {mp4_crf}")
    print(f"Text Size Category: {text_size_category}")
    print(f"Skip Dark Frames: {skip_dark_frames}")
    if skip_dark_frames: print(f"  Dark Frame Threshold (sum_lit <=): {dark_frame_threshold}")
    # --- Print new exclude_months option ---
    if exclude_months:
        valid_exclude_months = [m for m in exclude_months if 1 <= m <= 12]
        if len(valid_exclude_months) != len(exclude_months):
            print(f"WARN: Invalid month numbers removed from exclusion list: {exclude_months}")
        exclude_months = valid_exclude_months # Use only valid months
        if exclude_months: # Check if list is not empty after validation
            print(f"Exclude Months from Animation: {sorted(exclude_months)}")
        else:
            print("Exclude Months from Animation: None (after validation)")
            exclude_months = None # Set back to None if list became empty
    else:
        print("Exclude Months from Animation: None")
    # --- End exclude_months print ---
    print(f"Add Progress Bar: {add_progress_bar}")
    if add_progress_bar: print(f"  Progress Bar Height: {progress_bar_height}px")
    if debug_masking: print("!!! Mask Debugging Enabled !!!")

    # --- Validate Inputs ---
    # (Existing validation for masking...)
    if mask_outside_country:
        if not GEOPANDAS_AVAILABLE: print("WARN: Geopandas missing. Disabling mask."); mask_outside_country=False
        elif not country_boundary_shapefile_path: print("WARN: SHP path missing. Disabling mask."); mask_outside_country=False
        elif not target_country_name: print("WARN: Target country name missing. Disabling mask."); mask_outside_country=False
        elif not os.path.exists(country_boundary_shapefile_path): print(f"WARN: SHP not found: {country_boundary_shapefile_path}. Disabling mask."); mask_outside_country=False

    # --- 1. Find/Sort Files & Get Raster Metadata ---
    search_pattern = os.path.join(input_folder, '*.tif')
    tif_files = glob.glob(search_pattern)
    if not tif_files: print(f"Error: No .tif files found in {input_folder}"); return
    file_date_pairs = []
    for f in tif_files:
        date = get_date_from_filename(f)
        if date: file_date_pairs.append((date, f))
    if not file_date_pairs: print(f"Error: Could not parse dates from any filenames in {input_folder}"); return
    file_date_pairs.sort(); sorted_files = [f[1] for f in file_date_pairs]; sorted_dates = [f[0] for f in file_date_pairs]
    date_labels = [d.strftime('%Y-%m') for d in sorted_dates]
    print(f"Found and sorted {len(sorted_files)} TIFF files.")

    # --- Global Variables & Load Raster Metadata ---
    raster_transform = None; raster_shape = None; raster_crs = None;
    country_mask_array = None; country_mask_available = False
    try:
        with rasterio.open(sorted_files[0]) as src:
            raster_transform = src.transform; raster_shape = src.shape; raster_crs = src.crs
            if not raster_crs: print(f"WARNING: Raster {os.path.basename(sorted_files[0])} has no CRS. Masking might fail.")
            print(f"Raster Properties: Shape={raster_shape}, CRS={raster_crs}")
    except Exception as e: print(f"ERROR reading metadata from {os.path.basename(sorted_files[0])}: {e}"); return

    # --- Load Font ---
    font = None; calculated_font_size = 10; ttf_font_found = False
    print("\nAttempting to load font...")
    base_size = max(12, int(raster_shape[0] / 60)); size_multiplier = {'small': 0.7, 'medium': 1.0, 'large': 1.4}.get(text_size_category.lower(), 1.0)
    target_font_size = int(base_size * size_multiplier)
    # (Font loading logic - unchanged)
    if font_path and os.path.exists(font_path):
        try: font = ImageFont.truetype(font_path, target_font_size); calculated_font_size = target_font_size; print(f"  Loaded specified font: {font_path} size {calculated_font_size}"); ttf_font_found = True
        except Exception as e: print(f"Warn: Failed to load specified font '{font_path}': {e}")
    if not ttf_font_found:
        preferred_fonts = ['DejaVuSans', 'Arial', 'LiberationSans-Regular', 'Helvetica', 'Verdana', 'arial']
        found_path = find_system_font(preferred_fonts)
        if found_path:
            try: font = ImageFont.truetype(found_path, target_font_size); calculated_font_size = target_font_size; print(f"  Loaded system font: {found_path} size {calculated_font_size}"); ttf_font_found = True
            except Exception as e: print(f"Warn: Failed to load system font '{found_path}': {e}")
    if not ttf_font_found:
        try: font = ImageFont.load_default(size=int(target_font_size * 0.8)); calculated_font_size = 10; print("  Warn: Loaded default PIL bitmap font.")
        except Exception as e: print(f"ERROR: Default font loading failed: {e}. Text overlay disabled."); font = None

    # --- Prepare Country Mask ---
    # (Masking logic - unchanged)
    if mask_outside_country:
        print(f"\nPreparing country mask for '{target_country_name}'...")
        try:
            world_gdf = gpd.read_file(country_boundary_shapefile_path)
            if debug_masking: print(f" DBG: Loaded SHP cols: {world_gdf.columns.to_list()}")
            possible_name_cols = ['NAME','ADMIN','SOVEREIGNT','name','admin','sovereignt','NAME_0','COUNTRY','Name','Country']
            target_geom = None; country_gdf = None; target_found = False
            for col in possible_name_cols:
                 if col in world_gdf.columns:
                      if world_gdf[col].dtype == 'object':
                          country_gdf_candidate = world_gdf[world_gdf[col].str.lower() == target_country_name.lower()]
                          if not country_gdf_candidate.empty: country_gdf = country_gdf_candidate; target_geom = country_gdf.geometry; print(f"  Found '{target_country_name}' in col '{col}'."); target_found = True; break
                      elif debug_masking: print(f"  DBG: Skip non-string col '{col}'.")
            if not target_found:
                 print(f"  WARN: Exact match for '{target_country_name}' not found. Trying partial...");
                 for col in possible_name_cols:
                     if col in world_gdf.columns and world_gdf[col].dtype == 'object':
                          country_gdf_candidate = world_gdf[world_gdf[col].str.contains(target_country_name, case=False, na=False)]
                          if not country_gdf_candidate.empty: country_gdf=country_gdf_candidate; target_geom=country_gdf.geometry; print(f"  WARN: Found partial match(es) in col '{col}': {country_gdf[col].tolist()}. Using."); target_found=True; break
            if not target_found: raise ValueError(f"Could not find country '{target_country_name}'")
            if debug_masking: print(f" DBG: Shapefile CRS: {country_gdf.crs}")
            if not raster_crs: print(" WARN: Raster CRS undefined. Assuming compatible mask.")
            elif country_gdf.crs != raster_crs: print(f"  Reprojecting country geometry..."); country_gdf=country_gdf.to_crs(raster_crs); target_geom=country_gdf.geometry
            print("  Rasterizing country polygon(s)...")
            geoms_to_rasterize=[(g,1) for g in target_geom if g is not None and g.is_valid]
            if len(geoms_to_rasterize)<len(target_geom): print(f" WARN: Skipped {len(target_geom)-len(geoms_to_rasterize)} invalid geometries.")
            if not geoms_to_rasterize: raise ValueError("No valid geometries found.")
            country_mask_array = rasterize(shapes=geoms_to_rasterize,out_shape=raster_shape,transform=raster_transform,fill=0,default_value=1,dtype='uint8')
            country_mask_array = country_mask_array.astype(bool); country_mask_available = True
            num_masked_pixels = np.sum(country_mask_array); total_pixels = country_mask_array.size
            if num_masked_pixels==0: print("  WARNING: Mask is entirely FALSE. Check CRS/name/SHP."); country_mask_available=False
            elif num_masked_pixels==total_pixels: print("  WARNING: Mask is entirely TRUE.")
            else: print(f"  Country mask created ({num_masked_pixels} inside / {total_pixels} total).")
            if debug_masking: u,c=np.unique(country_mask_array, return_counts=True); print(f"  DBG: Mask unique/counts: {dict(zip(u, c))}")
        except ImportError: print("ERROR: Geopandas required for masking."); mask_outside_country=False
        except Exception as e: print(f"ERROR creating country mask: {e}. Disabling masking."); mask_outside_country=False; country_mask_available=False

    # --- 2. Process Files: Calculate Multiple Statistics ---
    # (Pass 1 logic - unchanged: calculate stats for ALL files for CSV/Graph)
    monthly_metrics_list = []; all_lit_pixels_for_range = []
    max_raw_value_overall = -np.inf; min_raw_value_overall = np.inf
    print("\n--- Pass 1: Calculating Metrics & Determining Value Range ---")
    for i, filepath in enumerate(sorted_files):
        date = sorted_dates[i]; print(f"  Processing {os.path.basename(filepath)} ({date.strftime('%Y-%m')})...", end='\r')
        metrics = {'date': date}
        try:
            with rasterio.open(filepath) as src:
                nodata_val = src.nodata; raw_data = src.read(1).astype(np.float32);
                if nodata_val is not None: raw_data[raw_data == nodata_val] = np.nan
                with warnings.catch_warnings(): warnings.simplefilter("ignore", category=RuntimeWarning); current_max_raw = np.nanmax(raw_data); current_min_raw = np.nanmin(raw_data)
                if np.isfinite(current_max_raw) and current_max_raw > max_raw_value_overall: max_raw_value_overall = current_max_raw
                if np.isfinite(current_min_raw) and current_min_raw < min_raw_value_overall: min_raw_value_overall = current_min_raw
                nan_mask=np.isnan(raw_data); lit_pixel_mask=~nan_mask & (raw_data > 0); non_negative_mask=~nan_mask & (raw_data >= 0)
                if country_mask_available: final_lit_mask = country_mask_array & lit_pixel_mask; final_nonneg_mask = country_mask_array & non_negative_mask
                else: final_lit_mask = lit_pixel_mask; final_nonneg_mask = non_negative_mask
                lit_pixels_in_mask_values = raw_data[final_lit_mask]; count_lit = lit_pixels_in_mask_values.size; metrics['count_lit'] = count_lit
                if count_lit > 0:
                    with warnings.catch_warnings(): warnings.simplefilter("ignore", category=RuntimeWarning); metrics['sum_lit'] = np.nansum(lit_pixels_in_mask_values); metrics['mean_lit'] = np.nanmean(lit_pixels_in_mask_values); metrics['median_lit'] = np.nanmedian(lit_pixels_in_mask_values); metrics['max_lit'] = np.nanmax(lit_pixels_in_mask_values)
                    all_lit_pixels_for_range.append(lit_pixels_in_mask_values)
                else: metrics['sum_lit'] = 0.0; metrics['mean_lit'] = np.nan; metrics['median_lit'] = np.nan; metrics['max_lit'] = np.nan
                nonneg_pixels_in_mask_values = raw_data[final_nonneg_mask]; count_nonneg = nonneg_pixels_in_mask_values.size; metrics['count_non_negative'] = count_nonneg
                if count_nonneg > 0:
                     with warnings.catch_warnings(): warnings.simplefilter("ignore", category=RuntimeWarning); metrics['sum_all'] = np.nansum(nonneg_pixels_in_mask_values); metrics['mean_all'] = np.nanmean(nonneg_pixels_in_mask_values); metrics['median_all'] = np.nanmedian(nonneg_pixels_in_mask_values)
                else: metrics['sum_all'] = 0.0; metrics['mean_all'] = np.nan; metrics['median_all'] = np.nan
        except Exception as e:
            print(f"\n  ERROR reading/processing {os.path.basename(filepath)}: {e}")
            metrics.setdefault('count_lit',0); metrics.setdefault('sum_lit',0.0); metrics.setdefault('mean_lit',np.nan); metrics.setdefault('median_lit',np.nan); metrics.setdefault('max_lit',np.nan)
            metrics.setdefault('count_non_negative',0); metrics.setdefault('sum_all',0.0); metrics.setdefault('mean_all',np.nan); metrics.setdefault('median_all',np.nan)
        monthly_metrics_list.append(metrics)
    print("\n--- Pass 1 Summary ---"); valid_months_count = sum(1 for m in monthly_metrics_list if m.get('count_lit', 0) > 0); print(f" Found lit pixels in {valid_months_count}/{len(sorted_files)} months.")
    if not np.isfinite(max_raw_value_overall): max_raw_value_overall=1.0;
    if not np.isfinite(min_raw_value_overall): min_raw_value_overall=0.0;
    print(f" Overall raw value range: {min_raw_value_overall:.4f} to {max_raw_value_overall:.4f}")
    vis_vmin, vis_vmax = 0.0, 1.0
    if all_lit_pixels_for_range:
        print(" Determining viz range using lit pixel percentiles...");
        try:
            all_lit_pixels_filt = [a for a in all_lit_pixels_for_range if isinstance(a,np.ndarray) and a.size>0]
            if all_lit_pixels_filt:
                 concatenated_data = np.concatenate(all_lit_pixels_filt); del all_lit_pixels_for_range
                 if concatenated_data.size > 0:
                     with warnings.catch_warnings(): warnings.simplefilter("ignore",category=RuntimeWarning); p_low=np.nanpercentile(concatenated_data,0.5); p_high=np.nanpercentile(concatenated_data,99.5)
                     if np.isfinite(p_low) and np.isfinite(p_high) and p_high>p_low: vis_vmin=max(0.0,p_low); vis_vmax=p_high; print(f" Using 0.5-99.5 lit range: {vis_vmin:.4f}-{vis_vmax:.4f}")
                     else: print(" Warn: Percentile calc failed. Fallback."); vis_vmin=max(0.0,min_raw_value_overall); vis_vmax=max(vis_vmin+1e-6,max_raw_value_overall); print(f" Using observed range: {vis_vmin:.4f}-{vis_vmax:.4f}")
                 else: print(" Warn: No valid lit data. Using default range [0,1].")
                 if 'concatenated_data' in locals(): del concatenated_data
            else: print(" Warn: No lit data arrays. Using default range [0,1].")
        except Exception as e: print(f" ERROR range calc: {e}. Using default [0,1]."); vis_vmin, vis_vmax = 0.0, 1.0
    else: print(" Warn: No lit pixels recorded. Using default [0,1]."); vis_vmin, vis_vmax = 0.0, 1.0
    if vis_vmax <= vis_vmin: vis_vmax = vis_vmin + 1.0
    if normalize_animation: print(f"--- Using NORMALIZED range: {vis_vmin:.4f}-{vis_vmax:.4f} ---"); norm=mcolors.Normalize(vmin=vis_vmin, vmax=vis_vmax)
    else: fixed_vmin=0.0; fixed_vmax=vis_vmax; print(f"--- Using FIXED scaling: {fixed_vmin:.4f}-{fixed_vmax:.4f} ---"); norm=mcolors.Normalize(vmin=fixed_vmin, vmax=fixed_vmax)

    # --- 3. Save Calculated Statistics to CSV ---
    # (CSV saving logic - unchanged: saves all calculated months)
    csv_filename=f"{output_path_base}_statistics.csv"; csv_headers=['Date','Mean_Lit','Median_Lit','Max_Lit','Sum_Lit','Count_Lit','Mean_AllNonNeg','Median_AllNonNeg','Sum_AllNonNeg','Count_AllNonNeg']
    print(f"\nSaving statistics to {csv_filename}...")
    try:
        with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter=csv.writer(csvfile); csvwriter.writerow(csv_headers); count_written=0
            for stats in monthly_metrics_list:
                def fmt(val, p=6): return f"{val:.{p}f}" if val is not None and np.isfinite(val) else ''
                row=[stats['date'].strftime('%Y-%m-%d'), fmt(stats.get('mean_lit')), fmt(stats.get('median_lit')), fmt(stats.get('max_lit')), fmt(stats.get('sum_lit')), stats.get('count_lit',0),
                     fmt(stats.get('mean_all')), fmt(stats.get('median_all')), fmt(stats.get('sum_all')), stats.get('count_non_negative',0)]; csvwriter.writerow(row); count_written += 1
            print(f"  Successfully wrote {count_written} rows to CSV.")
    except Exception as e: print(f"  ERROR writing CSV: {e}")

    # --- 4. Prepare Frames ---
    frames = []
    skipped_dark_count = 0
    skipped_month_count = 0 # <<< New counter for month exclusion
    cmap = cm.get_cmap(cmap_name)
    print("\n--- Pass 2: Creating animation frames ---")
    if len(outside_mask_color)==4: outside_mask_color_np=np.array(outside_mask_color[:4])/255.0
    else: print(f"Warn: Invalid outside_mask_color {outside_mask_color}. Using transparent."); outside_mask_color_np = np.array([0.0,0.0,0.0,0.0])
    total_files_for_progress = len(sorted_files)

    for i, filepath in enumerate(sorted_files):
        current_date = sorted_dates[i]
        current_month = current_date.month
        current_date_str = date_labels[i]
        print(f"  Processing frame {i+1}/{total_files_for_progress} ({current_date_str})...", end='\r')

        # --- Check 1: Manual Month Exclusion ---
        if exclude_months and current_month in exclude_months:
            skipped_month_count += 1
            # Optional: Print only once per excluded month for less verbose output
            # if current_month not in getattr(create_nightlight_timelapse_and_graph, '_printed_skip_months', set()):
            #      print(f"\n  Skipping month {current_month} ({current_date_str}) based on exclusion list.", " "*20)
            #      if not hasattr(create_nightlight_timelapse_and_graph, '_printed_skip_months'):
            #          create_nightlight_timelapse_and_graph._printed_skip_months = set()
            #      create_nightlight_timelapse_and_graph._printed_skip_months.add(current_month)
            continue # Skip to the next file

        # --- Check 2: Skip Dark Frames (only if not already skipped by month) ---
        if skip_dark_frames:
            try:
                current_sum_lit = monthly_metrics_list[i].get('sum_lit', np.nan)
                if np.isfinite(current_sum_lit) and current_sum_lit <= dark_frame_threshold:
                    skipped_dark_count += 1
                    # Optional: Print less verbosely
                    # if not getattr(create_nightlight_timelapse_and_graph, '_printed_dark_skip', False):
                    #     print(f"\n  Skipping dark frame ({current_date_str}), sum_lit <= {dark_frame_threshold:.4f}", " "*20)
                    #     create_nightlight_timelapse_and_graph._printed_dark_skip = True
                    continue # Skip to the next file
            except IndexError: print(f"\nWarn: Metrics index {i} missing for dark skip check.")
            except Exception as e: print(f"\nWarn: Error checking dark skip for frame {i+1}: {e}.")

        # --- If not skipped, proceed to create the frame ---
        try:
            with rasterio.open(filepath) as src:
                nodata_val=src.nodata; data=src.read(1).astype(np.float32)
                if nodata_val is not None: data[data == nodata_val] = np.nan
                processed_data = data.copy(); non_positive_mask = (~np.isnan(processed_data)) & (processed_data <= 0)
                processed_data[non_positive_mask] = norm.vmin; nan_mask_frame = np.isnan(processed_data); processed_data[nan_mask_frame] = norm.vmin
                normalized_data = norm(np.clip(processed_data, norm.vmin, norm.vmax)); rgba_image = cmap(normalized_data)
                rgba_image[nan_mask_frame, 3] = 0.0
                if mask_outside_country and country_mask_available:
                    outside_pixels_mask = ~country_mask_array
                    if np.any(outside_pixels_mask):
                        if outside_mask_color_np[3] == 0.0: rgba_image[outside_pixels_mask, 3] = 0.0
                        else: rgba_image[outside_pixels_mask, :] = outside_mask_color_np
                pil_image = Image.fromarray((rgba_image * 255).astype(np.uint8), 'RGBA'); draw = ImageDraw.Draw(pil_image)
                img_width, img_height = pil_image.size

                # Progress Bar
                if add_progress_bar:
                    try:
                        progress_ratio=(i+1)/total_files_for_progress; bar_fill_width=int(img_width*progress_ratio)
                        bar_y_start=img_height-progress_bar_height
                        draw.rectangle([(0, bar_y_start), (img_width, img_height)], fill=progress_bar_color_bg)
                        if bar_fill_width > 0: draw.rectangle([(0, bar_y_start), (bar_fill_width, img_height)], fill=progress_bar_color_fg)
                    except Exception as pb_err:
                        if i == 0: print(f"\nWarn: Error drawing progress bar: {pb_err}")

                # Text Overlays
                if font:
                    padding = max(5, int(calculated_font_size * 0.4))
                    # Watermark
                    if watermark_text:
                        try:
                            try: draw.text((padding, padding), watermark_text, fill=(255, 255, 255, 180), font=font)
                            except AttributeError: draw.text((padding, padding), watermark_text, fill=(255, 255, 255, 180), font=font)
                        except Exception as text_err:
                            if i == 0: print(f"\nWarn: Watermark draw error: {text_err}")
                    # Date
                    try:
                        try:
                            date_bbox = draw.textbbox((0, 0), current_date_str, font=font, anchor='lt')
                            text_width = date_bbox[2]-date_bbox[0]; text_height = date_bbox[3]-date_bbox[1]
                            pos_x = img_width - text_width - padding
                            pos_y = img_height - text_height - padding - (progress_bar_height if add_progress_bar else 0)
                            draw.text((pos_x, max(0, pos_y)), current_date_str, fill=(255, 255, 255, 200), font=font)
                        except AttributeError: # Fallback
                             est_width=int(len(current_date_str)*calculated_font_size*0.6); est_height=int(calculated_font_size*1.2)
                             pos_x = img_width - est_width - padding
                             pos_y = img_height - est_height - padding - (progress_bar_height if add_progress_bar else 0)
                             draw.text((pos_x, max(0, pos_y)), current_date_str, fill=(255, 255, 255, 200), font=font)
                    except Exception as text_err:
                         if i == 0: print(f"\nWarn: Date draw error: {text_err}")

                frames.append(np.array(pil_image))
        except Exception as e:
            print(f"\nERROR creating frame {i+1} for {os.path.basename(filepath)}: {e}")

    print("\nFinished creating frames.")
    # --- Updated summary ---
    if exclude_months: print(f"  Skipped {skipped_month_count} frames due to manual month exclusion.")
    if skip_dark_frames: print(f"  Skipped {skipped_dark_count} frames due to dark frame threshold.")
    final_frame_count = len(frames)
    total_skipped = skipped_month_count + skipped_dark_count
    print(f"  Total frames generated for animation: {final_frame_count} (out of {total_files_for_progress} input files, {total_skipped} skipped)")
    # --- End updated summary ---

    # --- 5. Create Animation ---
    if frames:
        output_filename_anim=f"{output_path_base}.{output_format.lower()}"; print(f"\nSaving animation: {output_filename_anim}...")
        frame_duration_ms = max(10, int(1000 / fps))
        try:
            if output_format.lower()=='gif': imageio.mimsave(output_filename_anim, frames, format='GIF', duration=frame_duration_ms, loop=0); print(" GIF saved.")
            elif output_format.lower()=='mp4': print(" INFO: Saving MP4..."); imageio.mimsave(output_filename_anim, frames, format='FFMPEG', fps=fps, output_params=['-vcodec', 'libx264', '-crf', str(mp4_crf), '-preset', 'medium', '-pix_fmt', 'yuva420p']); print(f" MP4 saved (CRF={mp4_crf}).")
            else: imageio.mimsave(output_filename_anim, frames, format=output_format, fps=fps); print(f" Animation saved ('{output_format}').")
        except Exception as e: print(f"\nError saving animation: {e}")
    elif total_skipped == total_files_for_progress:
         print("\nNo frames generated because all input files were skipped (due to month/darkness filters). Skipping animation.")
    else:
        print("\nNo frames generated (check for errors during frame creation). Skipping animation.")

    # --- 6. Create and Save Graph ---
    # (Graphing logic - unchanged: uses all data from monthly_metrics_list)
    output_filename_graph=f"{output_path_base}_graph.png"; print(f"\nGenerating statistics graph: {output_filename_graph}...")
    plot_metrics = {'mean_lit': {'label': 'Mean Radiance (Lit Pixels)', 'color': 'tab:blue', 'marker': 'o'}, 'sum_lit': {'label': 'Sum of Radiance (Lit Pixels)', 'color': 'tab:red', 'marker': 's'}}
    plt.style.use('seaborn-v0_8-darkgrid'); fig, ax1 = plt.subplots(figsize=(15, 7)); ax2 = ax1.twinx(); lines = []; labels = []; axes = {'mean_lit': ax1, 'sum_lit': ax2}; plot_successful = False
    for metric_key, props in plot_metrics.items():
        plot_dates=[s['date'] for s in monthly_metrics_list if np.isfinite(s.get(metric_key, np.nan))]; plot_values=[s[metric_key] for s in monthly_metrics_list if np.isfinite(s.get(metric_key, np.nan))]
        print(f"  Graph points for '{metric_key}': {len(plot_dates)}")
        if len(plot_dates) > 1: current_ax=axes.get(metric_key, ax1); line, = current_ax.plot(plot_dates, plot_values, marker=props.get('marker','.'), linestyle='-', markersize=5, color=props.get('color', None), label=props.get('label', metric_key)); lines.append(line); labels.append(props.get('label', metric_key)); current_ax.set_ylabel(props.get('label', metric_key), color=props.get('color', 'black')); current_ax.tick_params(axis='y', labelcolor=props.get('color', 'black')); current_ax.set_ylim(bottom=0); plot_successful = True
        elif len(plot_dates) == 1: current_ax=axes.get(metric_key, ax1); current_ax.plot(plot_dates, plot_values, marker=props.get('marker', 'o'), markersize=6, linestyle='', color=props.get('color', None), label=props.get('label', metric_key) + " (single pt)"); lines.append(current_ax.get_lines()[-1]); labels.append(props.get('label', metric_key) + " (single pt)"); current_ax.set_ylabel(props.get('label', metric_key), color=props.get('color', 'black')); current_ax.tick_params(axis='y', labelcolor=props.get('color', 'black')); current_ax.set_ylim(bottom=0); plot_successful = True
        else: print(f"  No valid data for '{metric_key}'.")
    if plot_successful:
        ax1.xaxis.set_major_locator(mdates.YearLocator(base=1)); ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y')); ax1.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=[1,7])); ax1.xaxis.set_minor_formatter(mdates.DateFormatter('%b')); plt.setp(ax1.xaxis.get_minorticklabels(), rotation=45, ha='right'); plt.setp(ax1.xaxis.get_majorticklabels(), rotation=0, ha='center')
        ax1.set_xlabel("Time"); ax1.set_title(graph_title); fig.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=len(labels), frameon=False); fig.tight_layout(rect=[0, 0.05, 1, 1])
        try: plt.savefig(output_filename_graph, dpi=300, bbox_inches='tight'); print("  Graph saved successfully.")
        except Exception as e: print(f"ERROR saving graph: {e}")
        plt.close(fig)
    else: print(f"  No valid data for plotting metrics. Skipping graph generation.")

    print("\nProcessing finished.")



In [8]:

# --- Example Usage (replace with your actual paths and settings) ---
if __name__ == '__main__':
    print("Running example usage...")

    # --- Configuration ---
    INPUT_DIR = r'C:/Users/rodri/My Drive/GEE_NightLights_Cuba'    # REQUIRED
    OUTPUT_BASE = r'C:/Users/rodri/Desktop/Nightlights/Results/Cuba/Cuba'   # REQUIRED
    SHAPEFILE_PATH = "C:/Users/rodri/Desktop/NIghtlights/Natural_Earth_Level_0/ne_10m_admin_0_countries.shp" # REQUIRED if masking
    COUNTRY_NAME = "Cuba"            # REQUIRED if masking

    # Optional settings
    MASK_COUNTRY = True
    OUTPUT_FMT = 'mp4'
    ANIM_FPS = 6
    NORMALIZE = False
    CMAP = 'plasma'
    MP4_QUALITY = 25
    TEXT_SIZE = 'medium'
    FONT_OVERRIDE = None
    WATERMARK = "ANAPLIAN.com Generated: " + datetime.now().strftime('%Y-%m-%d') # Disable watermark
    GRAPH_PLOT_TITLE = f"Night Light Trends: {COUNTRY_NAME}" if MASK_COUNTRY and COUNTRY_NAME else "Night Light Trends"
    DEBUG_MODE = False

    # Skip/Exclude settings
    SKIP_DARK = False            # Don't skip based on darkness
    DARK_THRESHOLD = 0.0
    #EXCLUDE_ANIM_MONTHS = [5,6,7,8] 

    SHOW_PROGRESS_BAR = True

    # --- Create output directory ---
    output_dir = os.path.dirname(OUTPUT_BASE)
    if output_dir and not os.path.exists(output_dir):
        print(f"Creating output directory: {output_dir}"); os.makedirs(output_dir)

    # --- Basic Input Validation ---
    if not os.path.isdir(INPUT_DIR): print(f"ERROR: Input dir not found: {INPUT_DIR}")
    elif MASK_COUNTRY and (not SHAPEFILE_PATH or not os.path.exists(SHAPEFILE_PATH)): print(f"ERROR: Masking enabled, but SHP not found/set: {SHAPEFILE_PATH}")
    elif MASK_COUNTRY and not COUNTRY_NAME: print(f"ERROR: Masking enabled, but country name not set.")
    else:
        # --- Run the main function ---
        create_nightlight_timelapse_and_graph(
            input_folder=INPUT_DIR,
            output_path_base=OUTPUT_BASE,
            output_format=OUTPUT_FMT,
            cmap_name=CMAP,
            fps=ANIM_FPS,
            normalize_animation=NORMALIZE,
            mask_outside_country=MASK_COUNTRY,
            country_boundary_shapefile_path=SHAPEFILE_PATH,
            target_country_name=COUNTRY_NAME,
            outside_mask_color=(50, 50, 50, 255), # Transparent mask
            mp4_crf=MP4_QUALITY,
            text_size_category=TEXT_SIZE,
            font_path=FONT_OVERRIDE,
            graph_title=GRAPH_PLOT_TITLE,
            watermark_text=WATERMARK,
            skip_dark_frames=SKIP_DARK,
            dark_frame_threshold=DARK_THRESHOLD,
            exclude_months=EXCLUDE_ANIM_MONTHS, # <<< Pass the exclusion list
            add_progress_bar=SHOW_PROGRESS_BAR,
            debug_masking=DEBUG_MODE
        )
        print("\nExample script finished.")

Running example usage...
Creating output directory: C:/Users/rodri/Desktop/Nightlights/Results/Cuba
Starting analysis for folder: C:/Users/rodri/My Drive/GEE_NightLights_Cuba
Output base: C:/Users/rodri/Desktop/Nightlights/Results/Cuba/Cuba
Format: mp4, FPS: 6, Colormap: plasma
Normalize Animation Frames: False
Mask Outside Country: True
Target Country Name: 'Cuba'
MP4 CRF Value: 25
Text Size Category: medium
Skip Dark Frames: False
Exclude Months from Animation: [5, 6, 7, 8]
Add Progress Bar: True
  Progress Bar Height: 5px
Found and sorted 155 TIFF files.
Raster Properties: Shape=(769, 2410), CRS=EPSG:4326

Attempting to load font...
  Found font 'DejaVuSans' by name.
  Loaded system font: DejaVuSans size 12

Preparing country mask for 'Cuba'...
  Found 'Cuba' in col 'NAME'.
  Rasterizing country polygon(s)...
  Country mask created (475394 inside / 1853290 total).

--- Pass 1: Calculating Metrics & Determining Value Range ---
  Processing Cuba_VIIRS_2025_02.tif (2025-02)...
--- Pass

C:\Users\rodri\AppData\Local\Temp\ipykernel_9372\64218847.py:331: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(cmap_name)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2410, 769) to (2416, 784) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


  Processing frame 155/155 (2025-02)...
Finished creating frames.
  Skipped 52 frames due to manual month exclusion.
  Total frames generated for animation: 103 (out of 155 input files, 52 skipped)

Saving animation: C:/Users/rodri/Desktop/Nightlights/Results/Cuba/Cuba.mp4...
 INFO: Saving MP4...
 MP4 saved (CRF=25).

Generating statistics graph: C:/Users/rodri/Desktop/Nightlights/Results/Cuba/Cuba_graph.png...
  Graph points for 'mean_lit': 155
  Graph points for 'sum_lit': 155
  Graph saved successfully.

Processing finished.

Example script finished.
